In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression

import pandoc

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

import math

import statsmodels.formula.api as smf

from sklearn.metrics import roc_curve, roc_auc_score
from scipy.stats import chi2
from statsmodels.stats.diagnostic import linear_harvey_collier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.api import het_breuschpagan

import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.outliers_influence import summary_table
from statsmodels.stats.diagnostic import linear_reset

from statsmodels.stats.diagnostic import lilliefors
from statsmodels.stats.diagnostic import het_white

from sklearn.metrics import roc_auc_score, roc_curve
from statsmodels.stats.diagnostic import het_breuschpagan

from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.api import het_white

from statsmodels.stats.diagnostic import linear_harvey_collier


from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.api import het_white

from statsmodels.stats.diagnostic import het_white
import os


In [2]:
stats_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/Go to Ground Analysis/ufc_fight_stats.csv'

results_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/Go to Ground Analysis/ufc_fight_results.csv'

stats_df = pd.read_csv(stats_path)

results_df = pd.read_csv(results_path)


In [33]:
# Hypothesis is that strikers with more of a grappling base (e.g. bjj blackbelt or wrestling) have a greater finish rate because they are more likely to go to the ground





array(['Tatsuro Taira', 'HyunSung Park', 'Mateusz Rebecki', ...,
       'David Levicki', 'Ray Wizard', 'Sean Daugherty'], dtype=object)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from rapidfuzz import fuzz
fighter_names =  stats_df['FIGHTER'].unique()
def search_wikipedia_page(fighter_name):
    search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={fighter_name}&format=json"
    try:
        r = requests.get(search_url, timeout=10)
        data = r.json()
        pages = data.get('query', {}).get('search', [])
        best_match = None
        highest_score = 0
        for page in pages[:3]:  # check top 3 results
            title = page['title']
            score = fuzz.token_sort_ratio(fighter_name.lower(), title.lower())
            if score > highest_score and score > 70:  # threshold to avoid bad matches
                best_match = title
                highest_score = score
        if best_match:
            url = f"https://en.wikipedia.org/wiki/{best_match.replace(' ', '_')}"
            return url
    except Exception as e:
        print(f"Error searching Wikipedia for {fighter_name}: {e}")
    return None

def extract_infobox(soup):
    info = {}
    infobox = soup.find('table', {'class': 'infobox'})
    if not infobox:
        return info
    for row in infobox.find_all('tr'):
        header = row.find('th')
        data = row.find('td')
        if header and data:
            key = header.get_text(strip=True).lower()
            value = data.get_text(" ", strip=True).lower()
            info[key] = value
    return info

def parse_belt_rank(text):
    if not text:
        return None
    belts = ['black belt', 'brown belt', 'purple belt', 'blue belt', 'white belt', 'red belt', 'dan']
    found = []
    for belt in belts:
        if belt in text:
            found.append(belt)
    dan_match = re.findall(r'(\d+)(st|nd|rd|th)? dan', text)
    if dan_match:
        found.extend([f"{m[0]} dan" for m in dan_match])
    if found:
        return ", ".join(found)
    return None

def check_olympic(text, sport):
    if not text:
        return 'No'
    if 'olympic' in text and sport in text:
        return 'Yes'
    return 'No'

def check_master_of_sport(text):
    if not text:
        return 'No'
    if 'master of sport' in text:
        return 'Yes'
    return 'No'

def get_wrestling_bjj_info(fighter_name, url):
    try:
        r = requests.get(url, timeout=15)
        soup = BeautifulSoup(r.text, 'html.parser')

        infobox_data = extract_infobox(soup)
        full_text = soup.get_text(separator=' ', strip=True).lower()

        bjj_belt_rank = None
        judo_belt_rank = None
        master_of_sport = 'No'
        olympic_wrestler = 'No'
        olympic_judo = 'No'
        wrestling_background = []
        bjj_background = []
        other_styles = []

        style_text = ""
        for key in infobox_data:
            if 'style' in key or 'martial art' in key or 'rank' in key or 'occupation' in key:
                style_text += " " + infobox_data[key]

        bjj_belt_rank = parse_belt_rank(style_text)
        judo_belt_rank = parse_belt_rank(style_text)  # you can refine logic here if needed

        master_of_sport = check_master_of_sport(style_text)
        olympic_wrestler = check_olympic(style_text, 'wrestling')
        olympic_judo = check_olympic(style_text, 'judo')

        sentences = re.split(r'(?<=[.!?]) +', full_text)
        wrestling_keywords = ['wrestling', 'wrestler', 'ncaa wrestling', 'folkstyle wrestling', 'freestyle wrestling', 'wrestled']
        bjj_keywords = ['brazilian jiu-jitsu', 'bjj', 'black belt', 'brown belt', 'purple belt', 'blue belt', 'white belt']
        other_styles_keywords = ['boxing', 'muay thai', 'kickboxing', 'karate', 'taekwondo', 'sambo', 'judo']

        for sentence in sentences:
            lower_sent = sentence.lower()
            if any(word in lower_sent for word in wrestling_keywords):
                wrestling_background.append(sentence.strip())
            if any(word in lower_sent for word in bjj_keywords):
                bjj_background.append(sentence.strip())
            if any(word in lower_sent for word in other_styles_keywords):
                other_styles.append(sentence.strip())

        return {
            'fighter': fighter_name,
            'bjj_belt_rank': bjj_belt_rank,
            'judo_belt_rank': judo_belt_rank,
            'master_of_sport': master_of_sport,
            'olympic_wrestler': olympic_wrestler,
            'olympic_judo': olympic_judo,
            'wrestling_background': " | ".join(wrestling_background[:3]),
            'bjj_background': " | ".join(bjj_background[:3]),
            'other_styles_summary': " | ".join(other_styles[:3]),
            'wiki_url': url
        }
    except Exception as e:
        print(f"Error scraping {fighter_name} at {url}: {e}")
        return None

def scrape_fighters_background(fighter_names, limit=None):
    results = []
    missing_fighters = []
    count = 0
    for name in fighter_names:
        print(f"Processing: {name}")
        wiki_url = search_wikipedia_page(name)
        if wiki_url:
            info = get_wrestling_bjj_info(name, wiki_url)
            if info:
                results.append(info)
            else:
                missing_fighters.append(name)
        else:
            print(f"No Wikipedia page found for {name}")
            missing_fighters.append(name)

        count += 1
        time.sleep(1)  # polite delay

        if limit and count >= limit:
            break

    df = pd.DataFrame(results)
    return df, missing_fighters

if __name__ == "__main__":
    # Example: replace this with your actual fighter names list
    # from your stats_df['FIGHTER'].unique()

    df_background, missing = scrape_fighters_background(fighter_names, limit=None)

    print(f"Scraped data for {len(df_background)} fighters.")
    if missing:
        print(f"Missing Wikipedia pages or scraping failed for {len(missing)} fighters:")
        print(missing)

    df_background.to_csv("ufc_fighters_bjj_wrestling_info.csv", index=False)
    print("Saved to ufc_fighters_bjj_wrestling_info.csv")


Processing: Tatsuro Taira
Processing: HyunSung Park


Saving CSV to: /Users/caseymoser/Desktop/UFC Analysis/UFC/Go to Ground Analysis
